In [1]:
import pandas as pd
import numpy as np
from scipy.io import mmwrite
from scipy.io import mmread
from scipy.sparse import csr_matrix
import requests
import implicit
from tqdm import tqdm

import gc
import json
import os

In [2]:
train_df = pd.read_pickle('train.pkl')
test_df = pd.read_pickle('test.pkl')
answer_df = pd.read_pickle('answer.pkl')

# for develop
answer_df = answer_df.sample(1000)
sampled_actors = set(answer_df.actor_id)

test_df = test_df[test_df.actor_id.isin(sampled_actors)]

train_sampled_actors = set(train_df.actor_id.sample(10000))
train_df = train_df[
    (train_df.actor_id.isin(train_sampled_actors)) | (train_df.actor_id.isin(sampled_actors))
]

In [3]:
df = pd.concat([train_df, test_df])

In [4]:
train_df.head()

,repo_id,actor_id,type,count
4,158726460,7874649,CommitCommentEvent,1
10,843222,14262420,CommitCommentEvent,1
20,105830539,11878166,ReleaseEvent,1
25,16008423,47363620,CommitCommentEvent,2
41,168425992,24392180,PublicEvent,1


In [5]:
test_df.head()

,repo_id,actor_id,type,count
4311457,15045751,21,IssueCommentEvent,1
14561420,11593442,21,WatchEvent,1
19437081,169677297,21,WatchEvent,1
21169543,170729130,21,WatchEvent,1
35339359,28894617,21,PullRequestEvent,1


In [6]:
answer_df.head()

,actor_id,repo_answer
0,21,"[225696514, 207117992, 179149772, 178738556, 5..."
1,34,"[132129752, 116403104, 112112945, 99715035, 63..."
2,118,"[30353566, 215973962, 54346799, 19415064, 4105..."
3,147,"[172359440, 20929025]"
4,176,"[45307548, 54581275, 172359440, 101033179]"


In [7]:
len(train_df), len(test_df), len(answer_df)

(17080512, 998717, 82810)

**repo meta API**

In [8]:
GITHUB_KEY = "/Users/kakao/practical_statistics/recohub/credentials/github.txt"
if os.path.exists(GITHUB_KEY):
    with open(GITHUB_KEY, 'r') as f:
        api_key = f.readline()        
        headers = {"Authorization": "bearer " + api_key.strip()}
def repo_meta_api(item_id):
    GITHUB_URL = "https://api.github.com/repositories/%s"%item_id
    res = requests.get(GITHUB_URL, headers=headers)

    if res.status_code == 200:
#         print(json.dumps(json.loads(res.text),indent=2))
        print(json.loads(res.text)['full_name'])

**NDCG calculator**

In [9]:
def ndcg_calculator(ans, rec):
    ans = set(ans)
    icdg = sum((1.0 / np.log(i+2) for i in range(len(ans))))
    return sum([1.0 / np.log(idx+2) for idx, repo in enumerate(rec) if repo in ans]) / icdg

### Top recommend: baseline

In [10]:
top_repos = df.groupby('repo_id')['count'].sum().sort_values(ascending=False).index.tolist()

ndcgs_basic = []
for _, row in tqdm(answer_df.iterrows(), total=len(answer_df)):
    ndcgs_basic.append(ndcg_calculator(row.repo_answer, top_repos[:len(row.repo_answer)]))

100%|██████████| 82810/82810 [00:11<00:00, 7126.35it/s]


In [11]:
print("Top-repo mean nDCG: %.4f"%(sum(ndcgs_basic) / len(ndcgs_basic)))

Top-repo mean nDCG: 0.0094


### ALS

In [12]:
def ui_matrix_generator(df, k):
    ui_count = df.groupby(['actor_id', 'repo_id'])['count'].sum().reset_index(name='count')
    while True:
        prev = len(ui_count)

        user_count = ui_count.groupby('actor_id').size()
        ui_count = ui_count[ui_count.actor_id.isin(user_count[user_count>=k].index)]

        repo_count = ui_count.groupby('repo_id').size()
        ui_count = ui_count[ui_count.repo_id.isin(repo_count[repo_count>=k].index)]

        if len(ui_count) == prev:
            break
        else:
            print("%s row deleted"%(prev-len(ui_count)))
    print("#user: %s, #repo: %s"%(ui_count.actor_id.nunique(), ui_count.repo_id.nunique()))
    return ui_count

def scr_generator(df, ui_count):
    ui_count['count_log'] = (ui_count['count']+1).apply(np.log)
    uid_to_idx = {uid: idx for (idx, uid) in enumerate(ui_count.actor_id.unique().tolist())}
    iid_to_idx = {iid: idx for (idx, iid) in enumerate(ui_count.repo_id.unique().tolist())}
    row, col, dat = ui_count.actor_id.tolist(), ui_count.repo_id.tolist(), ui_count['count_log'].tolist()
    row = [uid_to_idx[r] for r in row]
    col = [iid_to_idx[c] for c in col]
    train_matrix = csr_matrix((dat, (row,col)), shape=(1 + np.max(row), 1 + np.max(col)))
    return uid_to_idx, iid_to_idx, train_matrix

def als_model_generator(df, k):

    ui_count = ui_matrix_generator(df, k)
    uid_to_idx, iid_to_idx, train_matrix = scr_generator(df, ui_count)

    idx2uid = {v:k for k, v in uid_to_idx.items()}
    idx2iid = {v:k for k, v in iid_to_idx.items()}

    # initialize a model
    model = implicit.als.AlternatingLeastSquares(factors=20)
    # train the model on a sparse matrix of item/user/confidence weights
    model.fit(train_matrix.T)
    return train_matrix, model, idx2iid, idx2uid

def als_rec_generator(df, train_matrix, test_df, model, idx2iid, idx2uid):
    iid2idx = {v:k for k, v in idx2iid.items()}
    uid2idx = {v:k for k, v in idx2uid.items()}

    df_rec = pd.DataFrame(columns = ['repo_seen', 'repo_recommend'])
    target_keys = set(uid2idx.keys())
    no_rec = 0
    for actor_id, group in tqdm(test_df.groupby('actor_id')): # it takes about 20 minutes
        if actor_id in target_keys: # if the user is ommited by K-core
            df_rec.loc[actor_id] = group.repo_id.unique(), [idx2iid[i[0]] for i in model.recommend(uid2idx[actor_id], train_matrix, N=group.repo_id.nunique())]
        else:
            no_rec += 1
            df_rec.loc[actor_id] = group.repo_id.unique(), top_repos[:group.repo_id.nunique()]
    df_rec['actor_id'] = df_rec.index
    
    print("Inference done")
    return df_rec, no_rec

In [13]:
%%time
train_matrix, model, idx2iid, idx2uid = als_model_generator(df, 5)
df_rec, no_rec = als_rec_generator(df, train_matrix, test_df, model, idx2iid, idx2uid)
answer_als = answer_df.merge(df_rec, on='actor_id', how='outer')

964090 row deleted
1006 row deleted
23 row deleted
#user: 554767, #repo: 39599


100%|██████████| 105341/105341 [19:24<00:00, 90.45it/s]


Inference done
CPU times: user 2h 33min, sys: 3min 7s, total: 2h 36min 8s
Wall time: 23min 46s


In [14]:
print("추천 못 받은 유저 수: %s(%.2f%%)"%(no_rec, 100 * no_rec / len(answer_als)))

추천 못 받은 유저 수: 63340(60.13%)


In [15]:
%%time
ndcgs = []
for _, row in tqdm(answer_als.iterrows(), total=len(answer_als)):
    if isinstance(row.repo_answer, float): ndcgs.append(0)

    if row.repo_recommend != None:
        ndcgs.append(ndcg_calculator(row.repo_answer, row.repo_recommend))
    else:
        # if no recs, user top as fallback
        ndcgs.append(ndcg_calculator(row.repo_answer, top_repos[:len(row.repo_answer)]))

 78%|███████▊  | 82354/105341 [00:16<00:04, 5292.81it/s]

TypeError: 'float' object is not iterable

In [16]:
print("ALS mean nDCG: %.4f"%(sum(ndcgs) / len(ndcgs)))

ALS mean nDCG: 0.0379


In [17]:
# tf related repos
uid2idx = {v:k for k, v in idx2uid.items()}
iid2idx = {v:k for k, v in idx2iid.items()}

sim_items = model.similar_items(iid2idx[45717250], 20)
for i in range(20):
    repo_meta_api(idx2iid[sim_items[i][0]])

tensorflow/tensorflow
LucaCappelletti94/pygifsicle
rthadur/recaptcha
liutongxuan/tensorflow
tensorflow-jenkins/tensorflow
Wind-Kyle/ai-course-fundamentals
ROCmSoftwarePlatform/horovod
Intel-tensorflow/tensorflow
YunchuZhang/multiworld
YunchuZhang/softlearning
bjpublic/raspberrypi
BogGyver/openpilot
matsengrp/vampire
robbert-harms/MDT
MokkeMeguru/glow-realnvp-tutorial
r-tensorflow/r-tensorflow
Shashankjain12/Udadelhi
nengo/nengo.github.io
karllessard/tensorflow-community
jcfszxc/Project


### ALS: k=1

In [18]:
%%time
train_matrix, model, idx2iid, idx2uid = als_model_generator(df, 1)
df_rec, no_rec = als_rec_generator(df, train_matrix, test_df, model, idx2iid, idx2uid)
answer_als = answer_df.merge(df_rec, on='actor_id', how='outer')

 78%|███████▊  | 82354/105341 [00:30<00:04, 5292.81it/s]

#user: 1053410, #repo: 43149



  4%|▍         | 4150/105341 [00:14<06:31, 258.52it/s]


  7%|▋         | 7572/105341 [00:29<07:21, 221.36it/s]


 10%|█         | 10720/105341 [00:43<07:14, 217.82it/s]


...

 99%|█████████▊| 103940/105341 [20:07<00:31, 44.41it/s]


 99%|█████████▉| 104630/105341 [20:23<00:16, 43.78it/s]


100%|█████████▉| 105320/105341 [20:39<00:00, 43.65it/s]


100%|██████████| 105341/105341 [20:40<00:00, 84.94it/s]


Inference done
CPU times: user 2h 46min 1s, sys: 3min 28s, total: 2h 49min 30s
Wall time: 25min 41s


In [19]:
print("추천 못 받은 유저 수: %s(%.2f%%)"%(no_rec, 100 * no_rec / len(answer_als)))

추천 못 받은 유저 수: 0(0.00%)


In [20]:
%%time
ndcgs = []
for _, row in tqdm(answer_als.iterrows(), total=len(answer_als)):
    if sum(np.isnan(row.repo_answer)>0): continue

    if row.repo_recommend != None:
        ndcgs.append(ndcg_calculator(row.repo_answer, row.repo_recommend))
    else:
        # if no recs, user top as fallback
        ndcgs.append(ndcg_calculator(row.repo_answer, top_repos[:len(row.repo_answer)]))


 46%|████▋     | 48787/105341 [00:13<00:14, 3833.97it/s]


 78%|███████▊  | 82429/105341 [00:22<00:05, 4375.29it/s]

TypeError: 'numpy.bool_' object is not iterable

In [21]:
print("ALS mean nDCG: %.4f"%(sum(ndcgs) / len(ndcgs)))

ALS mean nDCG: 0.0472


### BPR

In [22]:
from implicit.bpr import BayesianPersonalizedRanking

In [23]:
def bpr_model_generator(df, k):

    ui_count = ui_matrix_generator(df, k)
    uid_to_idx, iid_to_idx, train_matrix = scr_generator(df, ui_count)

    idx2uid = {v:k for k, v in uid_to_idx.items()}
    idx2iid = {v:k for k, v in iid_to_idx.items()}

    # initialize a model
    model = implicit.bpr.BayesianPersonalizedRanking(factors=20, iterations=20)
    # train the model on a sparse matrix of item/user/confidence weights
    model.fit(train_matrix.T)
    return train_matrix, model, idx2iid, idx2uid

def bpr_rec_generator(df, train_matrix, test_df, model, idx2iid, idx2uid):
    iid2idx = {v:k for k, v in idx2iid.items()}
    uid2idx = {v:k for k, v in idx2uid.items()}

    df_rec = pd.DataFrame(columns = ['repo_seen', 'repo_recommend'])
    target_keys = set(uid2idx.keys())
    no_rec = 0
    for actor_id, group in tqdm(test_df.groupby('actor_id')): # it takes about 20 minutes
        if actor_id in target_keys: # if the user is ommited by K-core
            df_rec.loc[actor_id] = group.repo_id.unique(), [idx2iid[i[0]] for i in model.recommend(uid2idx[actor_id], train_matrix, N=group.repo_id.nunique())]
        else:
            no_rec += 1
            df_rec.loc[actor_id] = group.repo_id.unique(), top_repos[:group.repo_id.nunique()]
    df_rec['actor_id'] = df_rec.index
    
    print("Inference done")
    return df_rec, no_rec

In [24]:
%%time
train_matrix, model, idx2iid, idx2uid = bpr_model_generator(df, 5)
df_rec, no_rec = bpr_rec_generator(df, train_matrix, test_df, model, idx2iid, idx2uid)
answer_als = answer_df.merge(df_rec, on='actor_id', how='outer')


 78%|███████▊  | 82429/105341 [00:33<00:05, 4375.29it/s]

964090 row deleted
1006 row deleted
23 row deleted
#user: 554767, #repo: 39599




  0%|          | 0/105341 [00:00<?, ?it/s]

  0%|          | 28/105341 [00:00<06:25, 273.40it/s]

  0%|          | 63/105341 [00:00<06:01, 291.47it/s]

  0%|          | 98/105341 [00:00<05:45, 304.64it/s]

  0%|          | 126/105341 [00:00<05:55, 295.85it/s]

  0%|          | 159/105341 [00:00<05:44, 305.18it/s]

  0%|          | 193/105341 [00:00<05:35, 313.18it/s]

  0%|          | 222/105341 [00:00<05:49, 300.83it/s]

  0%|          | 259/105341 [00:00<05:29, 318.68it/s]

  0%|          | 290/105341 [00:00<05:37, 311.33it/s]

  0%|          | 321/105341 [00:01<05:42, 306.48it/s]

  0%|          | 352/105341 [00:01<05:54, 295.83it/s]

  0%|          | 382/105341 [00:01<06:13, 281.29it/s]

  0%|          | 411/105341 [00:01<06:13, 281.14it/s]

  0%|          | 440/105341 [00:01<06:40, 261.74it/s]

  0%|          | 476/105341 [00:01<06:09, 283.80it/s]

  0%|          | 513/105341 [00:01<05:44, 304.61it/s]

  1%|          | 545/105341 [00:01<07:31, 232.33it/s]

  1%|          | 578/1

  4%|▍         | 4036/105341 [00:14<06:36, 255.49it/s]

  4%|▍         | 4062/105341 [00:14<06:35, 255.83it/s]

  4%|▍         | 4089/105341 [00:14<06:30, 259.45it/s]

  4%|▍         | 4117/105341 [00:14<06:23, 263.73it/s]

  4%|▍         | 4146/105341 [00:14<06:15, 269.16it/s]

  4%|▍         | 4174/105341 [00:14<06:14, 270.46it/s]

  4%|▍         | 4202/105341 [00:14<06:15, 269.30it/s]

  4%|▍         | 4229/105341 [00:14<06:20, 266.05it/s]

  4%|▍         | 4256/105341 [00:14<06:26, 261.85it/s]

  4%|▍         | 4283/105341 [00:14<06:26, 261.48it/s]

  4%|▍         | 4310/105341 [00:15<06:25, 261.83it/s]

  4%|▍         | 4337/105341 [00:15<06:23, 263.49it/s]

  4%|▍         | 4365/105341 [00:15<06:17, 267.37it/s]

  4%|▍         | 4395/105341 [00:15<06:06, 275.40it/s]

  4%|▍         | 4424/105341 [00:15<06:02, 278.27it/s]

  4%|▍         | 4452/105341 [00:15<06:04, 276.51it/s]

  4%|▍         | 4480/105341 [00:15<06:06, 275.19it/s]

  4%|▍         | 4508/105341 [00:15<06:06, 275.4

  7%|▋         | 7497/105341 [00:27<06:48, 239.61it/s]

  7%|▋         | 7521/105341 [00:27<06:48, 239.51it/s]

  7%|▋         | 7545/105341 [00:27<06:48, 239.13it/s]

  7%|▋         | 7569/105341 [00:28<06:48, 239.33it/s]

  7%|▋         | 7594/105341 [00:28<06:45, 241.27it/s]

  7%|▋         | 7620/105341 [00:28<06:37, 245.95it/s]

  7%|▋         | 7645/105341 [00:28<06:38, 245.23it/s]

  7%|▋         | 7670/105341 [00:28<06:40, 243.76it/s]

  7%|▋         | 7695/105341 [00:28<06:42, 242.56it/s]

  7%|▋         | 7721/105341 [00:28<06:36, 246.37it/s]

  7%|▋         | 7746/105341 [00:28<06:43, 241.96it/s]

  7%|▋         | 7771/105341 [00:28<07:06, 228.53it/s]

  7%|▋         | 7795/105341 [00:29<07:04, 229.99it/s]

  7%|▋         | 7819/105341 [00:29<07:09, 227.19it/s]

  7%|▋         | 7842/105341 [00:29<07:15, 223.98it/s]

  7%|▋         | 7865/105341 [00:29<07:20, 221.14it/s]

  7%|▋         | 7888/105341 [00:29<07:18, 222.30it/s]

  8%|▊         | 7911/105341 [00:29<07:18, 221.9

...

 98%|█████████▊| 103289/105341 [19:16<00:43, 47.04it/s]

 98%|█████████▊| 103294/105341 [19:16<00:43, 46.67it/s]

 98%|█████████▊| 103299/105341 [19:17<00:42, 47.50it/s]

 98%|█████████▊| 103304/105341 [19:17<00:43, 47.03it/s]

 98%|█████████▊| 103309/105341 [19:17<00:43, 47.22it/s]

 98%|█████████▊| 103314/105341 [19:17<00:42, 47.32it/s]

 98%|█████████▊| 103319/105341 [19:17<00:42, 47.54it/s]

 98%|█████████▊| 103324/105341 [19:17<00:42, 47.55it/s]

 98%|█████████▊| 103329/105341 [19:17<00:42, 47.38it/s]

 98%|█████████▊| 103334/105341 [19:17<00:42, 46.82it/s]

 98%|█████████▊| 103339/105341 [19:17<00:42, 46.77it/s]

 98%|█████████▊| 103344/105341 [19:17<00:42, 46.70it/s]

 98%|█████████▊| 103349/105341 [19:18<00:42, 46.40it/s]

 98%|█████████▊| 103354/105341 [19:18<00:43, 46.05it/s]

 98%|█████████▊| 103359/105341 [19:18<00:43, 46.00it/s]

 98%|█████████▊| 103364/105341 [19:18<00:42, 46.05it/s]

 98%|█████████▊| 103369/105341 [19:18<00:42, 46.23it/s]

 98%|█████████▊| 103374/105341 

 99%|█████████▊| 103945/105341 [19:31<00:30, 45.83it/s]

 99%|█████████▊| 103950/105341 [19:31<00:31, 44.53it/s]

 99%|█████████▊| 103955/105341 [19:31<00:30, 45.30it/s]

 99%|█████████▊| 103960/105341 [19:31<00:30, 45.73it/s]

 99%|█████████▊| 103965/105341 [19:31<00:30, 45.75it/s]

 99%|█████████▊| 103970/105341 [19:31<00:29, 45.96it/s]

 99%|█████████▊| 103975/105341 [19:31<00:29, 45.62it/s]

 99%|█████████▊| 103980/105341 [19:31<00:29, 46.65it/s]

 99%|█████████▊| 103985/105341 [19:31<00:28, 46.88it/s]

 99%|█████████▊| 103990/105341 [19:32<00:29, 46.13it/s]

 99%|█████████▊| 103995/105341 [19:32<00:29, 46.13it/s]

 99%|█████████▊| 104000/105341 [19:32<00:29, 46.04it/s]

 99%|█████████▊| 104005/105341 [19:32<00:29, 45.94it/s]

 99%|█████████▊| 104010/105341 [19:32<00:28, 46.57it/s]

 99%|█████████▊| 104015/105341 [19:32<00:28, 46.76it/s]

 99%|█████████▊| 104020/105341 [19:32<00:28, 46.32it/s]

 99%|█████████▉| 104025/105341 [19:32<00:29, 45.21it/s]

 99%|█████████▉| 104030/105341 

 99%|█████████▉| 104606/105341 [19:45<00:16, 44.09it/s]

 99%|█████████▉| 104611/105341 [19:45<00:16, 44.11it/s]

 99%|█████████▉| 104616/105341 [19:45<00:16, 43.77it/s]

 99%|█████████▉| 104621/105341 [19:45<00:16, 43.16it/s]

 99%|█████████▉| 104626/105341 [19:45<00:16, 43.82it/s]

 99%|█████████▉| 104631/105341 [19:45<00:15, 44.50it/s]

 99%|█████████▉| 104636/105341 [19:46<00:15, 45.41it/s]

 99%|█████████▉| 104641/105341 [19:46<00:15, 45.95it/s]

 99%|█████████▉| 104646/105341 [19:46<00:15, 46.19it/s]

 99%|█████████▉| 104651/105341 [19:46<00:15, 45.69it/s]

 99%|█████████▉| 104656/105341 [19:46<00:15, 45.40it/s]

 99%|█████████▉| 104661/105341 [19:46<00:15, 44.85it/s]

 99%|█████████▉| 104666/105341 [19:46<00:15, 44.64it/s]

 99%|█████████▉| 104671/105341 [19:46<00:14, 45.04it/s]

 99%|█████████▉| 104676/105341 [19:46<00:14, 45.36it/s]

 99%|█████████▉| 104681/105341 [19:47<00:14, 45.51it/s]

 99%|█████████▉| 104686/105341 [19:47<00:14, 45.83it/s]

 99%|█████████▉| 104691/105341 

100%|█████████▉| 105260/105341 [19:59<00:01, 41.59it/s]

100%|█████████▉| 105265/105341 [19:59<00:01, 42.73it/s]

100%|█████████▉| 105270/105341 [19:59<00:01, 40.62it/s]

100%|█████████▉| 105275/105341 [20:00<00:01, 40.07it/s]

100%|█████████▉| 105280/105341 [20:00<00:01, 40.23it/s]

100%|█████████▉| 105285/105341 [20:00<00:01, 38.91it/s]

100%|█████████▉| 105290/105341 [20:00<00:01, 39.88it/s]

100%|█████████▉| 105295/105341 [20:00<00:01, 40.04it/s]

100%|█████████▉| 105300/105341 [20:00<00:01, 40.37it/s]

100%|█████████▉| 105305/105341 [20:00<00:00, 40.68it/s]

100%|█████████▉| 105310/105341 [20:00<00:00, 40.69it/s]

100%|█████████▉| 105315/105341 [20:00<00:00, 42.86it/s]

100%|█████████▉| 105321/105341 [20:01<00:00, 44.77it/s]

100%|█████████▉| 105326/105341 [20:01<00:00, 45.07it/s]

100%|█████████▉| 105331/105341 [20:01<00:00, 45.43it/s]

100%|█████████▉| 105336/105341 [20:01<00:00, 45.85it/s]

100%|██████████| 105341/105341 [20:01<00:00, 87.67it/s]

Inference done
CPU times: user 2h 38min 27s, sys: 3min 20s, total: 2h 41min 48s
Wall time: 26min 9s


In [25]:
%%time
ndcgs = []
for _, row in tqdm(answer_als.iterrows(), total=len(answer_als)):
    if sum(np.isnan(row.repo_answer)>0): continue

    if row.repo_recommend != None:
        ndcgs.append(ndcg_calculator(row.repo_answer, row.repo_recommend))
    else:
        # if no recs, user top as fallback
        ndcgs.append(ndcg_calculator(row.repo_answer, top_repos[:len(row.repo_answer)]))



  0%|          | 0/105341 [00:00<?, ?it/s]

  0%|          | 89/105341 [00:00<01:58, 889.25it/s]

  0%|          | 435/105341 [00:00<01:31, 1144.23it/s]

  1%|          | 766/105341 [00:00<01:13, 1423.64it/s]

  1%|          | 1101/105341 [00:00<01:00, 1719.74it/s]

  1%|▏         | 1434/105341 [00:00<00:51, 2010.91it/s]

  2%|▏         | 1762/105341 [00:00<00:45, 2273.81it/s]

  2%|▏         | 2103/105341 [00:00<00:40, 2525.22it/s]

  2%|▏         | 2441/105341 [00:00<00:37, 2731.80it/s]

  3%|▎         | 2786/105341 [00:00<00:35, 2909.59it/s]

  3%|▎         | 3146/105341 [00:01<00:33, 3086.64it/s]

  3%|▎         | 3501/105341 [00:01<00:31, 3211.64it/s]

  4%|▎         | 3840/105341 [00:01<00:31, 3243.79it/s]

  4%|▍         | 4185/105341 [00:01<00:30, 3300.82it/s]

  4%|▍         | 4535/105341 [00:01<00:30, 3356.92it/s]

  5%|▍         | 4893/105341 [00:01<00:29, 3419.23it/s]

  5%|▍         | 5244/105341 [00:01<00:29, 3445.69it/s]

  5%|▌         | 5592/105341 [00:01<00:28, 3442

 44%|████▍     | 46734/105341 [00:12<00:14, 3925.61it/s]

 45%|████▍     | 47127/105341 [00:13<00:15, 3871.80it/s]

 45%|████▌     | 47515/105341 [00:13<00:14, 3874.17it/s]

 45%|████▌     | 47903/105341 [00:13<00:14, 3862.32it/s]

 46%|████▌     | 48290/105341 [00:13<00:14, 3841.49it/s]

 46%|████▌     | 48684/105341 [00:13<00:14, 3868.87it/s]

 47%|████▋     | 49072/105341 [00:13<00:14, 3833.57it/s]

 47%|████▋     | 49463/105341 [00:13<00:14, 3855.51it/s]

 47%|████▋     | 49861/105341 [00:13<00:14, 3890.56it/s]

 48%|████▊     | 50258/105341 [00:13<00:14, 3912.19it/s]

 48%|████▊     | 50657/105341 [00:13<00:13, 3933.76it/s]

 48%|████▊     | 51052/105341 [00:14<00:13, 3937.96it/s]

 49%|████▉     | 51451/105341 [00:14<00:13, 3952.23it/s]

 49%|████▉     | 51847/105341 [00:14<00:13, 3950.85it/s]

 50%|████▉     | 52249/105341 [00:14<00:13, 3970.93it/s]

 50%|████▉     | 52667/105341 [00:14<00:13, 4031.33it/s]

 50%|█████     | 53071/105341 [00:14<00:13, 4004.47it/s]

 51%|█████    

TypeError: 'numpy.bool_' object is not iterable

In [26]:
print("BPR mean nDCG: %.4f"%(sum(ndcgs) / len(ndcgs)))

BPR mean nDCG: 0.0238




 78%|███████▊  | 82408/105341 [00:39<00:05, 4326.34it/s]